# Opening a new Italian restaurant in London, UK
by Aria Kumar

In [1]:
!conda install -c conda-forge geopy --yes 
!pip install geopy
!conda install -c conda-forge geocoder --yes  
!pip install geocoder
!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
!pip install beautifulsoup4 
!pip install lxml 
!pip install split

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None) #If set to None, the number of items to be printed is unlimited
pd.set_option('display.max_rows', None)    ##If set to None, the number of items to be printed is unlimited

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

from bs4 import BeautifulSoup
import string

from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib as plt
from matplotlib import pyplot

print('Libraries imported.')

Libraries imported.


In [3]:
import requests 

url = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
response = requests.get(url)  
print(response)  
print(response.status_code)
if 'json' in response.headers.get('Content-Type'):
    js = response.json()
else:
    print('Response content is not in JSON format.')
    js = 'spam'

<Response [200]>
200
Response content is not in JSON format.


In [4]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_areas_of_London').text


In [6]:
soup=BeautifulSoup(response,'lxml')


In [7]:
print(soup.title.text)

List of areas of London - Wikipedia


In [8]:
table = soup.find('table',class_='wikitable sortable')
table.tr.text 

'\nLocation\nLondon\xa0borough\nPost town\nPostcode\xa0district\nDial\xa0code\nOS grid ref\n'

In [9]:
headers=table.tr.text.split("\n")
headers

['',
 'Location',
 'London\xa0borough',
 'Post town',
 'Postcode\xa0district',
 'Dial\xa0code',
 'OS grid ref',
 '']

In [10]:
del headers[0]
del headers[-1]
headers

['Location',
 'London\xa0borough',
 'Post town',
 'Postcode\xa0district',
 'Dial\xa0code',
 'OS grid ref']

In [11]:
headers=[i.split()[0] for i in headers]
headers

['Location', 'London', 'Post', 'Postcode', 'Dial', 'OS']

In [12]:
headers[2]="Post town"
headers[5]="OS grid ref"
headers[4]="Dial code"
headers

['Location', 'London', 'Post town', 'Postcode', 'Dial code', 'OS grid ref']

In [13]:
table = soup.find('table',class_='wikitable sortable')


In [14]:
print(table.find_all('tr')[0])
print(table.find_all('tr')[0].text)
print(table.find_all('tr')[1]) 
print(table.find_all('tr')[1].text)

<tr>
<th>Location</th>
<th>London borough</th>
<th>Post town</th>
<th>Postcode district</th>
<th>Dial code</th>
<th>OS grid ref
</th></tr>

Location
London borough
Post town
Postcode district
Dial code
OS grid ref

<tr>
<td><a href="/wiki/Abbey_Wood" title="Abbey Wood">Abbey Wood</a></td>
<td>Bexley,  Greenwich <sup class="reference" id="cite_ref-mills1_7-0"><a href="#cite_note-mills1-7">[7]</a></sup></td>
<td>LONDON</td>
<td>SE2</td>
<td>020</td>
<td><span class="plainlinks nourlexpansion" style="white-space: nowrap"><a class="external text" href="https://tools.wmflabs.org/geohack/en/51.48648031512;0.10859224316653_region:GB_scale:25000?pagename=List_of_areas_of_London">TQ465785</a></span>
</td></tr>

Abbey Wood
Bexley,  Greenwich [7]
LONDON
SE2
020
TQ465785



In [15]:
import pandas as pd
row=[]
for tr in table.find_all('tr'):
    data = (tr.find_all('td'))
    row.append([i.text.strip() for i in data])

df1=pd.DataFrame(data=row)
df1=df1.iloc[1:]
df1.columns=headers
df1.reset_index(drop=True,inplace=True)
df1.head()

,Location,London,Post town,Postcode,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [16]:
lista = pd.read_html("https://en.wikipedia.org/wiki/List_of_areas_of_London",header=0)
type(lista)

list

In [17]:
df1= lista[1]

In [18]:
df1.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [19]:
print(df1.columns)

df1.columns = ['Location','Borough','Post_Town','Postcode','DialCode','OS_grid_ref']
print(df1.columns)

Index(['Location', 'London borough', 'Post town', 'Postcode district',
       'Dial code', 'OS grid ref'],
      dtype='object')
Index(['Location', 'Borough', 'Post_Town', 'Postcode', 'DialCode',
       'OS_grid_ref'],
      dtype='object')


In [20]:
for index,value in enumerate(df1['Borough']):
    
    df1.at[index,'Borough']=value.split('[')[0]  
df1['Borough'].head()

0                Bexley, Greenwich 
1    Ealing, Hammersmith and Fulham
2                           Croydon
3                           Croydon
4                            Bexley
Name: Borough, dtype: object

In [21]:
df1['Borough'] =  df1['Borough'].apply(lambda x: x.split(",")[0])

In [22]:
df1.head()

,Location,Borough,Post_Town,Postcode,DialCode,OS_grid_ref
0,Abbey Wood,Bexley,LONDON,SE2,020,TQ465785
1,Acton,Ealing,LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [23]:
df1['Postcode'] =  df1['Postcode'].apply(lambda x: x.split(",")[0])
df1['Post_Town'] =  df1['Post_Town'].apply(lambda x: x.split(",")[0])

In [24]:
df1.head()

,Location,Borough,Post_Town,Postcode,DialCode,OS_grid_ref
0,Abbey Wood,Bexley,LONDON,SE2,020,TQ465785
1,Acton,Ealing,LONDON,W3,020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,BEXLEY,DA5,020,TQ478728


In [25]:
df1.drop(columns=['Post_Town','DialCode','OS_grid_ref'],inplace=True)
df1.head()

,Location,Borough,Postcode
0,Abbey Wood,Bexley,SE2
1,Acton,Ealing,W3
2,Addington,Croydon,CR0
3,Addiscombe,Croydon,CR0
4,Albany Park,Bexley,DA5


In [26]:
df1.dtypes

Location    object
Borough     object
Postcode    object
dtype: object

In [27]:
df1 = df1.astype(str)
df1.dtypes


Location    object
Borough     object
Postcode    object
dtype: object

In [28]:
df1.sort_values(by='Postcode',ascending=True,inplace=True)
df1.reset_index(drop=True,inplace=True)
df1.head()

,Location,Borough,Postcode
0,Bromley,Bromley,BR1
1,Downham,Lewisham,BR1
2,Sundridge,Bromley,BR1
3,Widmore (also Widmore Green),Bromley,BR1
4,Plaistow,Bromley,BR1


In [29]:
df1.shape

(533, 3)

In [30]:
lista2 = pd.read_html("https://propertydata.co.uk/cities/london",header=0)
df2=lista2[0]
df2.head()

,Area,Avg yield,Avg price,£/sqft,5yr +/-,Explore data
0,BR1,3.5%,"£432,337",£461,+22%,Explore data
1,BR2,3.2%,"£490,587",£469,+23%,Explore data
2,BR3,3.6%,"£456,716",£488,+23%,Explore data
3,BR4,-,"£615,937",£456,+21%,Explore data
4,BR5,3.2%,"£446,841",£422,+23%,Explore data


In [31]:
df2.drop(columns=['Avg yield', 'Avg price', '5yr +/-', 'Explore data'],inplace=True)
df2.head()

,Area,£/sqft
0,BR1,£461
1,BR2,£469
2,BR3,£488
3,BR4,£456
4,BR5,£422


In [32]:
df2['£/sqft'] =  df2['£/sqft'].apply(lambda x: x.split("£")[-1])
df2.head()

,Area,£/sqft
0,BR1,461
1,BR2,469
2,BR3,488
3,BR4,456
4,BR5,422


In [33]:
df2.columns=['Postcode','£/sqft']
df2.head()

,Postcode,£/sqft
0,BR1,461
1,BR2,469
2,BR3,488
3,BR4,456
4,BR5,422


In [34]:
df=pd.merge(df1, df2, how='inner', left_on='Postcode', right_on='Postcode')
df

,Location,Borough,Postcode,£/sqft
0,Bromley,Bromley,BR1,461
1,Downham,Lewisham,BR1,461
2,Sundridge,Bromley,BR1,461
3,Widmore (also Widmore Green),Bromley,BR1,461
4,Plaistow,Bromley,BR1,461
5,Hayes,Bromley,BR2,469
6,Southborough,Bromley,BR2,469
7,Leaves Green,Bromley,BR2,469
8,Keston,Bromley,BR2,469
9,Bromley Common,Bromley,BR3,488


In [35]:
import geopy
address="London,UK"
geolocator = Nominatim(user_agent="London_explorer")
lat_lng_coords = None

while(lat_lng_coords is None):
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lat_lng_coords=[latitude,longitude]

print('The geograpical coordinate of London are {}, {}.'.format(lat_lng_coords[0], lat_lng_coords[1]))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [44]:
import geocoder
Latitude_list=[]
Longitude_list=[]
for i in range(df.shape[0]):
    address='{}, London,UK'.format(df.at[i,'Postcode']) 
    g = geocoder.arcgis(address)
    
    Latitude_list.append(g.latlng[0])
    Longitude_list.append(g.latlng[1])

In [45]:
df['Latitude']=Latitude_list
df['Longitude']=Longitude_list

In [46]:
df.head(10)

,Neighbourhood,Borough,Postcode,£/sqft,Latitude,Longitude
0,Bromley,Bromley,BR1,461,51.416710,0.009042
1,Downham,Lewisham,BR1,461,51.416710,0.009042
2,Sundridge,Bromley,BR1,461,51.416710,0.009042
3,Widmore (also Widmore Green),Bromley,BR1,461,51.416710,0.009042
4,Plaistow,Bromley,BR1,461,51.416710,0.009042
5,Hayes,Bromley,BR2,469,51.506420,-0.127210
6,Southborough,Bromley,BR2,469,51.506420,-0.127210
7,Leaves Green,Bromley,BR2,469,51.506420,-0.127210
8,Keston,Bromley,BR2,469,51.506420,-0.127210
9,Bromley Common,Bromley,BR3,488,51.415095,-0.035403


In [47]:
df=df.rename(columns = {'Location':'Neighbourhood'})

In [48]:
df.head()

,Neighbourhood,Borough,Postcode,£/sqft,Latitude,Longitude
0,Bromley,Bromley,BR1,461,51.41671,0.009042
1,Downham,Lewisham,BR1,461,51.41671,0.009042
2,Sundridge,Bromley,BR1,461,51.41671,0.009042
3,Widmore (also Widmore Green),Bromley,BR1,461,51.41671,0.009042
4,Plaistow,Bromley,BR1,461,51.41671,0.009042


In [49]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng,postcode, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Postcode'],df['Borough'], df['Neighbourhood']):
    label = '{},{}, {}'.format(postcode,neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [90]:
map_london.save('map_london.html')

In [50]:
CLIENT_ID = '0RYP2WXGOQPAHFLHED5BKLTUVVAN01X0A1ZE4YMJ5QBQMWPI' # your Foursquare ID
CLIENT_SECRET = '1XIDUSR2VXZFJOONIQZ3RAV5FDQ5OAWT5ZINMPKPAO50EJ5R' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0RYP2WXGOQPAHFLHED5BKLTUVVAN01X0A1ZE4YMJ5QBQMWPI
CLIENT_SECRET:1XIDUSR2VXZFJOONIQZ3RAV5FDQ5OAWT5ZINMPKPAO50EJ5R


In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):  
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):  
        
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']   
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results]) 
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 

In [53]:
london_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
          
                                longitudes=df['Longitude'])

Bromley
Downham
Sundridge
Widmore (also Widmore Green)
Plaistow
Hayes
Southborough
Leaves Green
Keston
Bromley Common
Bickley
Beckenham
Elmers End
Eden Park
West Wickham
Coney Hall
St Paul's Cray
Petts Wood
Orpington
St Mary Cray
Derry Downs
Chelsfield
Downe
Locksbottom
Pratt's Bottom
Goddington
Hazelwood
Elmstead
Chislehurst
Woodside
Forestdale
Waddon
Croydon
Coombe
New Addington
Addiscombe
Addington
Shirley
Selsdon
Sanderstead
South Croydon
Mitcham
Old Coulsdon
Coulsdon
Thornton Heath
Purley
Kenley
Riddlesdown
Crayford
Barnes Cray
Blackfen
Lamorbey
Welling
East Wickham
Belvedere
Lessness Heath
Dartford
Bexley (also Old Bexley, Bexley Village)
Albany Park
Barnehurst
North End
Northumberland Heath
Erith
Slade Green
Colyers
Mile End
Ratcliff
Stepney
Spitalfields
Wapping
Whitechapel
Shadwell
Leyton
Lea Bridge
Wanstead
Cann Hall
Leytonstone
Snaresbrook
Little Ilford
Manor Park
Plaistow
West Ham
Leamouth
Cubitt Town
Isle of Dogs
Millwall
Limehouse
Poplar
Canary Wharf
Blackwall
Stratford
Ma

In [54]:
london_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hayes,51.50642,-0.12721,Corinthia Hotel,51.506607,-0.124460,Hotel
1,Hayes,51.50642,-0.12721,Trafalgar Square,51.507987,-0.128048,Plaza
2,Hayes,51.50642,-0.12721,East Trafalgar Square Fountain,51.508088,-0.127700,Fountain
3,Hayes,51.50642,-0.12721,Horse Guards Parade,51.504847,-0.126590,Plaza
4,Hayes,51.50642,-0.12721,ESPA Life at Corinthia,51.506402,-0.125114,Spa


In [55]:
london_venues.shape

(21349, 7)

In [57]:
london_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Abbey Wood,8,8,8,8,8,8
Acton,7,7,7,7,7,7
Addington,4,4,4,4,4,4
Addiscombe,4,4,4,4,4,4
Albany Park,100,100,100,100,100,100
Aldborough Hatch,100,100,100,100,100,100
Aldwych,100,100,100,100,100,100
Alperton,100,100,100,100,100,100
Anerley,6,6,6,6,6,6


In [51]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 315 uniques categories.


In [59]:
london_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hayes,51.50642,-0.12721,Corinthia Hotel,51.506607,-0.124460,Hotel
1,Hayes,51.50642,-0.12721,Trafalgar Square,51.507987,-0.128048,Plaza
2,Hayes,51.50642,-0.12721,East Trafalgar Square Fountain,51.508088,-0.127700,Fountain
3,Hayes,51.50642,-0.12721,Horse Guards Parade,51.504847,-0.126590,Plaza
4,Hayes,51.50642,-0.12721,ESPA Life at Corinthia,51.506402,-0.125114,Spa


In [64]:
london_venues.groupby(["Venue Category"]).count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,4,4,4,4,4,4
Afghan Restaurant,7,7,7,7,7,7
African Restaurant,6,6,6,6,6,6
American Restaurant,25,25,25,25,25,25
Antique Shop,1,1,1,1,1,1
Arcade,4,4,4,4,4,4
Arepa Restaurant,3,3,3,3,3,3
Argentinian Restaurant,32,32,32,32,32,32
Art Gallery,361,361,361,361,361,361


In [65]:
london_venues['Venue Category'].unique()

array(['Hotel', 'Plaza', 'Fountain', 'Spa', 'Tea Room',
       'Monument / Landmark', 'Outdoor Sculpture', 'Art Gallery',
       'Bookstore', 'Art Museum', 'Restaurant', 'Theater', 'Church',
       'Garden', 'History Museum', 'Pub', 'Hotel Bar', 'Thai Restaurant',
       'Wine Bar', 'English Restaurant', 'Indie Movie Theater',
       'Spanish Restaurant', 'North Indian Restaurant', 'Candy Store',
       'Italian Restaurant', 'Opera House', 'Café', 'Burger Joint',
       'Sandwich Place', 'Park', 'Boutique', 'Gift Shop', 'Coffee Shop',
       'Ice Cream Shop', 'Arts & Crafts Store', 'Bar', 'Liquor Store',
       'Greek Restaurant', 'Pizza Place', 'Mexican Restaurant',
       'Cocktail Bar', 'Gay Bar', 'Japanese Restaurant',
       'French Restaurant', 'Bakery', 'Steakhouse', 'Seafood Restaurant',
       'Ramen Restaurant', 'Asian Restaurant', 'Chinese Restaurant',
       'Lounge', 'Portuguese Restaurant', 'Lebanese Restaurant',
       'Pakistani Restaurant', 'Athletics & Sports', 'Train

In [67]:
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
london_onehot.head()

,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Recruiting Agency,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan

In [68]:
london_onehot['Neighbourhood'] = london_venues['Neighbourhood'] 
london_onehot.head()

,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Recruiting Agency,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan

In [69]:
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]
london_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Recruiting Agency,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports C

In [70]:
london_onehot.shape

(21349, 317)

In [71]:
london_grouped = london_onehot.groupby('Neighbourhood').mean().reset_index()
london_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Camera Store,Canal,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Recruiting Agency,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports C

In [72]:
london_grouped.shape

(500, 317)

In [73]:
len(london_grouped[london_grouped["Italian Restaurant"] > 0])

257

In [75]:
london_res = london_grouped[["Neighbourhood","Italian Restaurant"]]

In [76]:
london_res.head()

,Neighbourhood,Italian Restaurant
0,Abbey Wood,0.00
1,Acton,0.00
2,Addington,0.00
3,Addiscombe,0.00
4,Albany Park,0.02


In [77]:
kclusters = 3

london_clustering = london_res.drop(["Neighbourhood"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

In [79]:
london_merged = london_res.copy()

london_merged["Cluster Labels"] = kmeans.labels_

In [80]:
london_merged.rename(columns={"Neighbourhood": "Neighbourhood"}, inplace=True)
london_merged.head()

,Neighbourhood,Italian Restaurant,Cluster Labels
0,Abbey Wood,0.00,0
1,Acton,0.00,0
2,Addington,0.00,0
3,Addiscombe,0.00,0
4,Albany Park,0.02,0


In [81]:
london_merged = london_merged.join(df.set_index("Neighbourhood"), on="Neighbourhood")

print(london_merged.shape)
london_merged.head()

(506, 8)


,Neighbourhood,Italian Restaurant,Cluster Labels,Borough,Postcode,£/sqft,Latitude,Longitude
0,Abbey Wood,0.00,0,Bexley,SE2,383,51.492450,0.121270
1,Acton,0.00,0,Ealing,W3,678,51.513240,-0.267460
2,Addington,0.00,0,Croydon,CR0,432,51.384755,-0.051499
3,Addiscombe,0.00,0,Croydon,CR0,432,51.384755,-0.051499
4,Albany Park,0.02,0,Bexley,DA5,428,51.506420,-0.127210


In [82]:
print(london_merged.shape)
london_merged.sort_values(["Cluster Labels"], inplace=True)
london_merged

(506, 8)


,Neighbourhood,Italian Restaurant,Cluster Labels,Borough,Postcode,£/sqft,Latitude,Longitude
0,Abbey Wood,0.000000,0,Bexley,SE2,383,51.492450,0.121270
315,North Sheen,0.020000,0,Richmond upon Thames,TW9,784,51.506420,-0.127210
314,North Kensington,0.000000,0,Kensington and Chelsea,W10,948,51.523460,-0.213530
311,North End,0.020000,0,Bexley,DA8,350,51.506420,-0.127210
310,Norbury,0.000000,0,Croydon,SW16,547,51.418700,-0.127530
308,Noak Hill,0.020000,0,Havering,RM3,381,51.506420,-0.127210
307,Nine Elms,0.000000,0,Wandsworth,SW8,919,51.476230,-0.130560
306,Newington,0.000000,0,Southwark,SE1,955,51.499600,-0.096130
305,Newbury Park,0.020000,0,Redbridge,IG2,419,51.506420,-0.127210
304,New Southgate,0.000000,0,Barnet,N11,551,51.616310,-0.138390


In [83]:
kclusters = 3
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]  

markers_colors = []
for lat, lon,postcode, neigh, cluster in zip(london_merged['Latitude'], london_merged['Longitude'],london_merged['Postcode'],london_merged['Neighbourhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(postcode)+","+str(neigh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1], #if cluster is 0 then the 1- will lead to a color of #ff0000 in the rainbow list!
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)  #fill_opacity fills the circle.if it is zero the circle is similar to a donut.
       
map_clusters

In [84]:
map_clusters.save('map_clusters.html')

In [85]:
london_merged.loc[london_merged['Cluster Labels'] == 0]

,Neighbourhood,Italian Restaurant,Cluster Labels,Borough,Postcode,£/sqft,Latitude,Longitude
0,Abbey Wood,0.000000,0,Bexley,SE2,383,51.492450,0.121270
315,North Sheen,0.020000,0,Richmond upon Thames,TW9,784,51.506420,-0.127210
314,North Kensington,0.000000,0,Kensington and Chelsea,W10,948,51.523460,-0.213530
311,North End,0.020000,0,Bexley,DA8,350,51.506420,-0.127210
310,Norbury,0.000000,0,Croydon,SW16,547,51.418700,-0.127530
308,Noak Hill,0.020000,0,Havering,RM3,381,51.506420,-0.127210
307,Nine Elms,0.000000,0,Wandsworth,SW8,919,51.476230,-0.130560
306,Newington,0.000000,0,Southwark,SE1,955,51.499600,-0.096130
305,Newbury Park,0.020000,0,Redbridge,IG2,419,51.506420,-0.127210
304,New Southgate,0.000000,0,Barnet,N11,551,51.616310,-0.138390


In [86]:
london_merged.loc[london_merged['Cluster Labels'] == 1]

,Neighbourhood,Italian Restaurant,Cluster Labels,Borough,Postcode,£/sqft,Latitude,Longitude
488,Winchmore Hill,0.166667,1,Enfield,N21,545,51.635190,-0.093030
88,Chinbrook,0.125000,1,Lewisham,SE12,468,51.447590,0.013500
491,Woodford Green,0.200000,1,Redbridge,IG8,483,51.608869,0.015320
490,Woodford,0.200000,1,Redbridge,IG8,483,51.608869,0.015320
22,Barnes Cray,0.200000,1,Bexley,DA1,359,51.452068,0.172230
489,Wood Green,0.200000,1,Haringey,N22,589,51.602140,-0.112230
53,Bowes Park,0.200000,1,Haringey,N22,589,51.602140,-0.112230
106,Crayford,0.200000,1,Bexley,DA1,359,51.452068,0.172230
195,Hampton Wick,0.142857,1,Richmond upon Thames,KT1,611,51.351429,-0.249881
208,Harrow on the Hill,0.250000,1,Harrow,HA1,536,51.567891,-0.339483


In [87]:
london_merged.loc[london_merged['Cluster Labels'] == 2]

,Neighbourhood,Italian Restaurant,Cluster Labels,Borough,Postcode,£/sqft,Latitude,Longitude
173,Golders Green,0.074074,2,Barnet,NW11,718,51.576730,-0.196950
175,Gospel Oak,0.104167,2,Camden,NW5,878,51.555700,-0.139260
91,Chiswick,0.076923,2,Hounslow,W4,797,51.489440,-0.261940
458,Wandsworth,0.035714,2,Wandsworth,SW18,772,51.450660,-0.189970
179,Greenwich,0.027027,2,Greenwich,SE10,655,51.484540,0.002750
27,Bayswater,0.050000,2,Westminster,W2,"1,272",51.514940,-0.180480
26,Battersea,0.037037,2,Wandsworth,SW11,867,51.467600,-0.162900
180,Grove Park,0.085106,2,Lewisham,SE12,468,51.447590,0.013500
457,Walworth,0.031250,2,Southwark,SE17,768,51.487640,-0.095420
33,Bedford Park,0.076923,2,Ealing,W4,797,51.489440,-0.261940
